# Capstone Project - IBM Data Science

## Final Report

### Opening a pizza place in Oakland, CA

* Construct a dataframe of neighborhoods in Oakland, CA by scraping data from a Wiki page
* Obtain the geographical coordinates of the neighborhoods
* Use Foursquare API to obtain the venue data for the neighborhoods
* Explore and cluster the neighborhoods to determine the best cluster to open a new pizza place

## Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium
print("Libraries imported.")

Libraries imported.


## Scrap data from Wiki page into a DataFrame

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Oakland,_California").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
# create a new DataFrame from the list
ca_df = pd.DataFrame({"Neighborhood": neighborhoodList})

ca_df.head()

,Neighborhood
0,"List of neighborhoods in Oakland, California"
1,"Template:Oakland, California neighborhoods"
2,"Acorn, Oakland, California"
3,"Adams Point, Oakland, California"
4,"Allendale, Oakland, California"


In [7]:
ca_df.drop([0, 1], inplace=True)
ca_df.reset_index(drop=True, inplace=True)
ca_df.head()

,Neighborhood
0,"Acorn, Oakland, California"
1,"Adams Point, Oakland, California"
2,"Allendale, Oakland, California"
3,"Beulah Heights, California"
4,"Brookfield Village, Oakland, California"


In [8]:
# print the number of rows of the dataframe
ca_df.shape

(70, 1)

## Obtain geographical coordinates

In [9]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Oakland, CA'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
# call the function to get the coordinates
coords = [ get_latlng(neighborhood) for neighborhood in ca_df["Neighborhood"].tolist() ]

In [11]:
coords

[[37.80508000000003, -122.27306999999996],
 [37.8147716093137, -122.25515030543295],
 [37.784165066228354, -122.20339431845163],
 [33.82403000000005, -118.17632999999995],
 [37.733231021335335, -122.13779129475161],
 [37.8038002111481, -122.24625690217515],
 [37.80508000000003, -122.27306999999996],
 [37.80861253099133, -122.29884476882057],
 [37.80861253099133, -122.29884476882057],
 [37.74518000000006, -122.12605999999994],
 [37.799780000000055, -122.27276999999998],
 [37.80271000000005, -122.26772999999997],
 [37.863541262339666, -122.23486814256505],
 [37.838326008357235, -122.25365772822397],
 [37.81110000000007, -122.22758999999996],
 [37.80126000000007, -122.27497999999997],
 [37.80437000000006, -122.27079999999995],
 [37.82974000000007, -122.25571999999994],
 [37.82240525714764, -122.24466147069633],
 [37.76757332714819, -122.17755318577478],
 [37.74370239466514, -122.17675662771873],
 [37.78068255171434, -122.22430363375976],
 [37.80437000000006, -122.27079999999995],
 [37.812

In [12]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
ca_df['Latitude'] = df_coords['Latitude']
ca_df['Longitude'] = df_coords['Longitude']
ca_df.dropna()

,Neighborhood,Latitude,Longitude
0,"Acorn, Oakland, California",37.805080,-122.273070
1,"Adams Point, Oakland, California",37.814772,-122.255150
2,"Allendale, Oakland, California",37.784165,-122.203394
3,"Beulah Heights, California",33.824030,-118.176330
4,"Brookfield Village, Oakland, California",37.733231,-122.137791
5,"Brooklyn, California",37.803800,-122.246257
6,"Bushrod, Oakland, California",37.805080,-122.273070
7,"Campbell Village Court, Oakland, California",37.808613,-122.298845
8,"Campbell Village, Oakland, California",37.808613,-122.298845
9,Chabot Park,37.745180,-122.126060


In [14]:
#check
print(ca_df.shape)
ca_df.head()

(70, 3)


,Neighborhood,Latitude,Longitude
0,"Acorn, Oakland, California",37.805080,-122.273070
1,"Adams Point, Oakland, California",37.814772,-122.255150
2,"Allendale, Oakland, California",37.784165,-122.203394
3,"Beulah Heights, California",33.824030,-118.176330
4,"Brookfield Village, Oakland, California",37.733231,-122.137791


In [15]:
#save as csv
ca_df.to_csv("ca_df.csv", index=False)

## Create a map of Oakland with superimposed neighborhoods

In [16]:
# get the coordinates of Oakland

latitude = 37.804363
longitude = -122.271111
print('The geograpical coordinates of Oakland, CA: {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Oakland, CA: 37.804363, -122.271111.


In [17]:
# create map of San Fransisco using latitude and longitude values
map_ca = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(ca_df['Latitude'], ca_df['Longitude'], ca_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ca)  
    
map_ca

In [18]:
# save the map as HTML file
map_ca.save('map_ca.html')

## Explore the neighborhoods with Foursquare API

In [19]:
# define Foursquare Credentials and Version
CLIENT_ID = 'DHTNTMSZNLVLBSP1EH1N2KU4HQAWHMBIQPRHVVP52K5ZH4R0' # your Foursquare ID
CLIENT_SECRET = 'GABJLS0VQMKYX1O0MXJUPEIYB1VCRTFJ5WU2PUXUTZNLMYTC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DHTNTMSZNLVLBSP1EH1N2KU4HQAWHMBIQPRHVVP52K5ZH4R0
CLIENT_SECRET:GABJLS0VQMKYX1O0MXJUPEIYB1VCRTFJ5WU2PUXUTZNLMYTC


#### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [21]:

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(ca_df['Latitude'], ca_df['Longitude'], ca_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6136, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Acorn, Oakland, California",37.80508,-122.27307,Oaklandish,37.805075,-122.270726,Clothing Store
1,"Acorn, Oakland, California",37.80508,-122.27307,Cape & Cowl,37.806725,-122.272747,Comic Shop
2,"Acorn, Oakland, California",37.80508,-122.27307,Bar Shiru,37.806378,-122.270393,Bar
3,"Acorn, Oakland, California",37.80508,-122.27307,Fox Theater,37.808136,-122.270670,Music Venue
4,"Acorn, Oakland, California",37.80508,-122.27307,Woods Bar & Brewery,37.806889,-122.270415,Brewery


#### Let's check how many venues were returned for each neighborhood

In [23]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Acorn, Oakland, California",100,100,100,100,100,100
"Adams Point, Oakland, California",100,100,100,100,100,100
"Allendale, Oakland, California",100,100,100,100,100,100
"Beulah Heights, California",100,100,100,100,100,100
"Brookfield Village, Oakland, California",100,100,100,100,100,100
"Brooklyn, California",100,100,100,100,100,100
"Bushrod, Oakland, California",100,100,100,100,100,100
"Campbell Village Court, Oakland, California",61,61,61,61,61,61
"Campbell Village, Oakland, California",61,61,61,61,61,61


#### Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 254 uniques categories.


In [25]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Clothing Store', 'Comic Shop', 'Bar', 'Music Venue', 'Brewery',
       'Vegetarian / Vegan Restaurant', 'Mexican Restaurant', 'Café',
       'Dance Studio', 'Seafood Restaurant', 'Japanese Restaurant',
       'Taco Place', 'Brazilian Restaurant', 'Caribbean Restaurant',
       'Coffee Shop', 'Ice Cream Shop', 'Bagel Shop', 'Food Court',
       'Climbing Gym', 'Beer Bar', 'Burger Joint', 'Wine Bar',
       'New American Restaurant', 'Chinese Restaurant', 'Gay Bar',
       'Tiki Bar', 'Beer Garden', 'Bubble Tea Shop',
       'Gym / Fitness Center', 'Bakery', 'Farmers Market',
       'Sandwich Place', 'Nightclub', 'Dessert Shop',
       'Fried Chicken Joint', 'Tea Room', 'Salon / Barbershop', 'Museum',
       'Indian Restaurant', 'Indie Movie Theater', 'Cocktail Bar',
       'Dim Sum Restaurant', 'Plaza', 'Furniture / Home Store',
       'Yoga Studio', 'Jazz Club', 'American Restaurant', 'Sports Bar',
       'Dumpling Restaurant', 'Performing Arts Venue'], dtype=object)

## Analyse each neighborhood

In [26]:
# one hot encoding
ca_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ca_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ca_onehot.columns[-1]] + list(ca_onehot.columns[:-1])
ca_onehot = ca_onehot[fixed_columns]

print(ca_onehot.shape)
ca_onehot.head()

(6136, 255)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Amphitheater,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Library,College Theater,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Mountain,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Resort,Restaurant,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soba Restaurant,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Synagogue,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"Acorn, Oakland, California",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
ca_grouped = ca_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(ca_grouped.shape)
ca_grouped

(70, 255)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Amphitheater,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Casino,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Library,College Theater,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Mountain,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Resort,Restaurant,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soba Restaurant,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Synagogue,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"Acorn, Oakland, California",0.00,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.010000,0.040000,0.000000,0.080000,0.000000,0.000000,0.00,0.000000,0.020000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.020000,0.010000,0.020000,0.00,0.000000,0.000000,0.000000,0.030000,0.00,0.000000,0.00,0.010000,0.010000,0.00,0.040000,0.01,0.010000,0.01,0.08

In [28]:
len(ca_grouped[ca_grouped["Pizza Place"] > 0])

66

#### Create a new DataFrame for Pizza Place data only

In [29]:
ca_pp = ca_grouped[["Neighborhoods","Pizza Place"]]

In [30]:
ca_pp.head()

,Neighborhoods,Pizza Place
0,"Acorn, Oakland, California",0.01
1,"Adams Point, Oakland, California",0.01
2,"Allendale, Oakland, California",0.03
3,"Beulah Heights, California",0.04
4,"Brookfield Village, Oakland, California",0.03


## Cluster neighborhoods

In [32]:
# set number of clusters
kclusters = 3

ca_clustering = ca_pp.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ca_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 1, 1, 0, 0, 0, 0, 1], dtype=int32)

In [33]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
ca_merged = ca_pp.copy()

# add clustering labels
ca_merged["Cluster Labels"] = kmeans.labels_

In [34]:
ca_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
ca_merged.head()

,Neighborhood,Pizza Place,Cluster Labels
0,"Acorn, Oakland, California",0.01,0
1,"Adams Point, Oakland, California",0.01,0
2,"Allendale, Oakland, California",0.03,1
3,"Beulah Heights, California",0.04,1
4,"Brookfield Village, Oakland, California",0.03,1


In [35]:
# merge sf_grouped with sf_data to add latitude/longitude for each neighborhood
ca_merged = ca_merged.join(ca_df.set_index("Neighborhood"), on="Neighborhood")

print(ca_merged.shape)
ca_merged.head() # check the last columns!

(70, 5)


,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
0,"Acorn, Oakland, California",0.01,0,37.805080,-122.273070
1,"Adams Point, Oakland, California",0.01,0,37.814772,-122.255150
2,"Allendale, Oakland, California",0.03,1,37.784165,-122.203394
3,"Beulah Heights, California",0.04,1,33.824030,-118.176330
4,"Brookfield Village, Oakland, California",0.03,1,37.733231,-122.137791


In [36]:
# sort the results by Cluster Labels
print(ca_merged.shape)
ca_merged.sort_values(["Cluster Labels"], inplace=True)
ca_merged

(70, 5)


,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
0,"Acorn, Oakland, California",0.010000,0,37.805080,-122.273070
22,"Ghost Town, Oakland, California",0.010000,0,37.804370,-122.270800
23,"Glenview, Oakland, California",0.020000,0,37.812045,-122.244600
25,"Grand Lake, Oakland, California",0.020000,0,37.820661,-122.250515
26,"Grass Valley, Oakland, California",0.000000,0,37.751925,-122.120310
51,"Peralta Villa, Oakland, California",0.020000,0,37.819286,-122.287608
29,"Jack London District, Oakland, California",0.010000,0,37.795032,-122.276962
30,Jack London Square,0.010000,0,37.794880,-122.277030
31,Jingletown,0.010000,0,37.805080,-122.273070
32,"Lakeside Apartments District, Oakland, California",0.000000,0,37.805280,-122.263180


#### Finally, let's visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ca_merged['Latitude'], ca_merged['Longitude'], ca_merged['Neighborhood'], ca_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examine clusters

### Cluster 1

In [39]:
ca_merged.loc[ca_merged['Cluster Labels'] == 0]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
0,"Acorn, Oakland, California",0.010000,0,37.805080,-122.273070
22,"Ghost Town, Oakland, California",0.010000,0,37.804370,-122.270800
23,"Glenview, Oakland, California",0.020000,0,37.812045,-122.244600
25,"Grand Lake, Oakland, California",0.020000,0,37.820661,-122.250515
26,"Grass Valley, Oakland, California",0.000000,0,37.751925,-122.120310
51,"Peralta Villa, Oakland, California",0.020000,0,37.819286,-122.287608
29,"Jack London District, Oakland, California",0.010000,0,37.795032,-122.276962
30,Jack London Square,0.010000,0,37.794880,-122.277030
31,Jingletown,0.010000,0,37.805080,-122.273070
32,"Lakeside Apartments District, Oakland, California",0.000000,0,37.805280,-122.263180


### Cluster 2

In [40]:
ca_merged.loc[ca_merged['Cluster Labels'] == 1]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
56,"Ridgemont, Oakland, California",0.030303,1,37.784793,-122.168169
45,"Oakland Hills, Oakland, California",0.030000,1,37.822405,-122.244661
64,"Sobrante Park, Oakland, California",0.025316,1,37.727180,-122.175860
46,"Oakland Point, Oakland, California",0.030000,1,37.822405,-122.244661
62,"Seminary, Oakland, California",0.028169,1,37.764960,-122.199748
61,"Seminary Park, Oakland, California",0.028169,1,37.764960,-122.199748
53,"Piedmont Pines, Oakland, California",0.040000,1,37.826157,-122.252743
60,"Santa Fe, Oakland, California",0.030000,1,37.847094,-122.275608
59,San Pablo Arts District,0.040000,1,36.743335,-119.794678
52,"Piedmont Avenue (Oakland, California)",0.040000,1,37.826157,-122.252743


### Cluster 3

In [41]:
ca_merged.loc[ca_merged['Cluster Labels'] == 2]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
40,"Melrose, Oakland, California",0.060000,2,37.774774,-122.206668
66,"Trestle Glen, Oakland, California",0.070000,2,37.807882,-122.228447
65,"Temescal, Oakland, California",0.060000,2,37.835420,-122.262770
13,"Clinton, Oakland, California",0.050000,2,37.838326,-122.253658
19,"Eastmont, Oakland, California",0.047619,2,37.767573,-122.177553
17,Downtown Oakland,0.050000,2,37.829740,-122.255720
57,"Rockridge, Oakland, California",0.050000,2,37.844830,-122.244861
55,"Redwood Heights, Oakland, California",0.072165,2,37.795320,-122.198045
33,"Laurel, Oakland, California",0.052083,2,37.793328,-122.206958
47,"Oakmore, Oakland, California",0.064103,2,37.807435,-122.214700


#### Observations:


Cluster 1 is situated in the city centre of Oakland and has a low concentration of Pizza Places. Clusters 2 and 3 are situated around the outskirts of the city centre with Cluster 2 having an intermediate concentration of Pizza Places and Cluster 3 having a high concentration of Pizza Places. Neighborhoods in Cluster 1 represent potentially good locations for a new Pizza Place due to the lack of competition from other Pizza Places, however, being in the dense city centre may see threat from other food places which may exist in a higher concentration in these neighbourhoods. Neighbourhoods in Cluster 3 would not be recommended as good locations as these neighborhoods are already saturated with Pizza Places and so this project advises against this. Neighbourhoods in Cluster 2 may be the optimum locations for a new Pizza place as they are located away from the busy city centre, saturated with other food places, and also only have moderate concentrations of Pizza Places. Thus providing the opportunity for a new Pizza Place to be successful.